# Deploy R Shiny

### Introduction

This notebook shows how the R Shiny Application can be deployed. 

The notebook we will:
* Store Shiny assets into the same deployment space.
* Deploy Shiny assets as an app and view the dashboard.

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2019, 2020. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**

In [1]:
import os
import pandas as pd
import datetime
from ibm_watson_machine_learning import APIClient

token = os.environ['USER_ACCESS_TOKEN']
url= os.environ['RUNTIME_ENV_APSX_URL']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": url,
   "version": "3.5"
}

client = APIClient(wml_credentials) 

### Set up Deployment Space

The following code programmatically gets the deployment space and the model deployment details. 

In [2]:
SPACE_NAME = 'ClaimsLeakage'

# Check is the space already exists
space_uid = ''
for space in client.spaces.get_details()['resources']:
    if space['entity']['name'] == SPACE_NAME:
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)
        break

if space_uid == '':
    print("Deployment space",SPACE_NAME,"not found.")

# set this space as default space
client.set.default_space(space_uid)

'SUCCESS'

### User Inputs 

We need the Deployment IDs of the models that are used to make the predictions, namely, the relocation and the home purchase model.  The command below lists all deployments in the space.

In [3]:
client.deployments.list()

------------------------------------  ------------------------  -----  ------------------------
GUID                                  NAME                      STATE  CREATED
538c8434-2ee1-41e9-970d-208fbbe91775  claims_leakage_Shiny_App  ready  2021-04-28T17:28:46.890Z
9d5a2d3d-3a99-413d-9e7f-3a76956c9187  claims leakage DO model   ready  2021-04-28T16:40:39.752Z
28b8d326-c5da-4145-9279-49955f5904b2  claims leakage DO model   ready  2021-04-28T16:31:19.045Z
3c6a2dcd-9ec2-4918-ad75-4978d23ffd47  claims leakage DO model   ready  2021-04-28T16:19:39.313Z
ffc683b8-04dd-4e5e-b1c7-e7e3156dc48a  claims leakage DO model   ready  2021-04-28T09:23:40.108Z
57966280-43d8-4303-894b-2f1bd099250e  Pipeline_7_webservice     ready  2021-04-20T08:27:28.044Z
------------------------------------  ------------------------  -----  ------------------------


### User Input

#### Note: Specify below the hostname, username, password and functionid to be used in the RShiny deployment to access the CPD deployed function

In [4]:
hostname = 'https://...com'
username = 'insert_username' #e.g. mluser1
password = 'insert_password' #e.g. XXXXXX
functionID = 'claims_leakage_DO_model_GUID' #insert GUID of claims leakage DO model from client.deployments.list() in previous cell

In [5]:
# When running on Cloud click the option "Insert project token" - if not running on cloud project is setup below
try:
    project
except NameError:
    # READING AND WRITING PROJECT ASSETS
    import project_lib
    project = project_lib.Project() 

In [6]:
!ls /project_data/data_asset

app.zip				   solution_280421_092104.csv
claim.csv			   solution_280421_092107.csv
claims_leakage.csv		   solution_280421_161813.csv
current_assignments.csv		   solution_280421_161817.csv
handler.csv			   solution_280421_161819.csv
handler_list.csv		   solution_280421_161820.csv
leakage.csv			   solution_280421_163008.csv
model_template.py		   solution_280421_163009.csv
model_template.zip		   solution_280421_163011.csv
new_claim_cases.csv		   solution_280421_164017.csv
predictions_for_DO_production.csv  solution_280421_164020.csv
__pycache__			   solution_280421_164021.csv
regression_leakage_autoAI.csv	   solution_280421_164022.csv
solution_280421_091732.csv	   solution_balanced_11022021.json
solution_280421_091923.csv	   solution_base_11022021.json
solution_280421_091938.csv	   solution_sll_11022021.json
solution_280421_092013.csv	   solution_slr_11022021.json
solution_280421_092056.csv	   test_model_template.py


In [7]:
# Create a local copy of the RShiny deployment zip file ready for deployment
!rm -f app.zip
f = open('app.zip', "wb")
f.write(project.get_file('app.zip').getbuffer())
f.close()

In [8]:
!ls

app.zip


# Deploy Shiny App

In this section we will complete the steps to deploy a Shiny Dashboard in Cloud Pak for Data. The app can be deployed in a similar way to models and functions, using the watson_machine_learning_client package.

All of the files associated with the dashboard are contained in a zip file which is stored in data assets. If the user would like to make changes to the dashboard, they can download the zip from data assets and upload it in the RStudio IDE. 

In [9]:
r_shiny_deployment_name='claims_leakage_Shiny_App'

### Store the App

Create the associated metadata and store the dashboard zip file in the deployment space. 

In [10]:
# Meta_props to store assets in space 
meta_props = {
    client.shiny.ConfigurationMetaNames.NAME: "claims_leakage_Shiny_assets",
    client.shiny.ConfigurationMetaNames.DESCRIPTION: 'Store shiny assets in deployment space' # optional
}
app_details = client.shiny.store(meta_props, 'app.zip')

Creating Shiny asset...
SUCCESS


### Deploy the App

Create the metadata for the Shiny deployment by providing  name, description, R-Shiny options and Hardware specifications. R-Shiny configuration provides options on whom you want to share the dashboard with, they are 
* Anyone with the link 
* Authenticated users  
* Collaborators in this deployment space

In [11]:
# Deployment metadata.
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: r_shiny_deployment_name,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: 'Deploy Claimns Leakage dashboard',
    client.deployments.ConfigurationMetaNames.R_SHINY: { 'authentication': 'anyone_with_url' },
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S', 'num_nodes': 1}
}

# Create the deployment.
app_uid = client.shiny.get_uid(app_details)
rshiny_deployment = client.deployments.create(app_uid, deployment_meta_props)



#######################################################################################

Synchronous deployment creation for uid: '071063f3-f7eb-4e17-8084-29a2fb2c2157' started

#######################################################################################


initializing.......
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='3bede666-ae56-4e16-bf92-c720573e0ee2'
------------------------------------------------------------------------------------------------




In [12]:
rshiny_deployment

{'entity': {'asset': {'id': '071063f3-f7eb-4e17-8084-29a2fb2c2157'},
  'custom': {},
  'deployed_asset_type': 'r_shiny',
  'description': 'Deploy Claimns Leakage dashboard',
  'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b',
   'name': 'S',
   'num_nodes': 1},
  'name': 'claims_leakage_Shiny_App',
  'r_shiny': {'authentication': 'anyone_with_url'},
  'space_id': 'eb5e5a97-6539-405d-99f3-d5fe2a94f6db',
  'status': {'rshiny_url': {'url': 'https://internal-nginx-svc:12443/ml/v4/deployments/3bede666-ae56-4e16-bf92-c720573e0ee2/r_shiny'},
   'state': 'ready'}},
 'metadata': {'created_at': '2021-04-28T17:44:53.441Z',
  'description': 'Deploy Claimns Leakage dashboard',
  'id': '3bede666-ae56-4e16-bf92-c720573e0ee2',
  'modified_at': '2021-04-28T17:44:53.441Z',
  'name': 'claims_leakage_Shiny_App',
  'owner': '1000331017',
  'space_id': 'eb5e5a97-6539-405d-99f3-d5fe2a94f6db'}}

### Launch Shiny App
Now that the dashboard is deployed, it can be accessed through the web browser. The app URL can be found by navigating to the deployed app in the deployment space. 

Open the Navigation Menu, select **Deployments -> Spaces -> ClaimsLeakage -> Deployments -> claims_leakage_Shiny_App** to find the dashboard URL.

Alternatively, the path for the app URL can be found from the deployment metadata created in the previous cell. This path should be appended to the user's Cloud Pak for Data hostname to get the complete app URL. To get the path, run the cell below:

In [15]:
print(hostname+"/ml/v4/deployments/"+rshiny_deployment['metadata']['id'] + '/r_shiny' + '/app')

https://dse-accelerators-demo.cpolab.ibm.com/ml/v4/deployments/3bede666-ae56-4e16-bf92-c720573e0ee2/r_shiny/app


In [14]:
#!pip install pyhli